# Reason Why This Notebook Exists

### There are various notebooks on Kaggle describing how to get started with the image classificatin using pyutorch or Tensorflow to detect Covid-19 by analysing X-rays. However most of them are highly complicated or least they are complicated to me. So I decided to help out everyone by creating this ver simple starte notebook. It shows how you can train an EfficientNet model using the Kaggle data set and pytorch library.

# Introduction TO Covid-19 problem

# Importing Libraries

In [ ]:
# Libraries and dependencies needed for dicom visualation and model training
# !pip install torchsummary
# !pip install efficientnet_pytorch
# !pip install pylibjpeg pylibjpeg-libjpeg
# !conda install -c conda-forge gdcm -y
# !pip install python-gdcm
# !pip install pylibjpeg
# import pydicom
# !python -m pip uninstall numpy --yes
# !pip install numpy==1.19.2


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
!pip install efficientnet_pytorch
!pip install torchsummary
from torchsummary import summary
import PIL
import sys
import torch
from time import time
import torchvision
from PIL import Image
import torch.nn as nn
from torch.utils import data
from torch.autograd import Variable
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
# I have used Efficientnet3 pre-trained model for the classification task
from efficientnet_pytorch import EfficientNet
# !python -m pip uninstall numpy --yes
# !python -m pip uninstall pydicom --yes
# !pip install pydicom
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

from torch.autograd import Variable
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as om
import torchvision as tv
import torch.utils.data as dat

# I have used Efficientnet3 pre-trained model for the classification task

if torch.cuda.is_available():     # Make sure GPU is available
    dev = torch.device("cuda:0")
    kwar = {'num_workers': 8, 'pin_memory': True}
    cpu = torch.device("cpu")
else:
    print("Warning: CUDA not found, CPU only.")
    dev = torch.device("cpu")
    kwar = {}
    cpu = torch.device("cpu")

# Learning About Datasets

## Loading the training set image and study ids

In [ ]:
training_set_study = pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')
training_set_image_level = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')
training_set_study.head(100)

In [ ]:
# looking at the image_level_training_set.
training_set_image_level.head(100)

In [ ]:
# Changing the column values to allow for merging of two tables. The merged tables can then be used to extract image data.
training_set_image_level.id = training_set_image_level.id.str.replace('_image','')
training_set_image_level.head()
training_set_study.id = training_set_study.id.str.replace('_study','')
training_set_study.head()
# training_set_image_level.loc[training_set_image_level.StudyInstanceUID == '005057b3f880']

# Merging Two Training Set Tables
The StudyInstanceUID in train_image_level is same as the 'id' in train_study_level. Henceforth these two tables can be merged based on these two columns.

In [ ]:
combined_training_set = training_set_image_level.merge(training_set_study, left_on = 'StudyInstanceUID', right_on = 'id', how = 'inner', suffixes=('_image', 'study'))
combined_training_set.drop(columns = 'idstudy', inplace = True)
combined_training_set.head()


### Getting path of each image and adding it to the combined tables
<!--     Assumptions -->
1. Some image folders have multiple images however we have neglected that and instead only selected the first image in that folder.

In [ ]:
# Creating function to get absolute path.

def get_absolute_file_paths(x):
    path = '../input/siim-covid19-detection/train/'
    directory = os.path.join(path,x )
    all_abs_file_paths = []
    for dirpath,_,filenames in os.walk(directory):
        for f in filenames:
            all_abs_file_paths.append(os.path.abspath(os.path.join(dirpath, f)))
    return all_abs_file_paths
# get_absolute_file_paths(combined_training_set.StudyInstanceUID[0])

In [ ]:
combined_training_set['abs_image_path'] = combined_training_set.StudyInstanceUID.apply(get_absolute_file_paths) 
combined_training_set.head()

# Reading X-rays for image visualisation

## Function for reading dicom images

In [ ]:
# Function for reading .dicom files
# ----------> Copid from 'siim-cov19 efnb7+yolov5 [infer]' :) ~~~~~~~~ Thanks
def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

# Visualisation
### Reading 10 random images and showing there labels

In [ ]:
!pip install python-gdcm
!pip install pylibjpeg pylibjpeg-libjpeg
fig, axs = plt.subplots(nrows = 2, ncols = 5, sharex = True, sharey = True, figsize = (20, 10), tight_layout = True)
for ax, Index_num in zip(axs.reshape(-1), np.random.choice(range(len(combined_training_set)), 20, replace=False)):
    data = read_xray(combined_training_set.abs_image_path[Index_num][0])
    data = cv2.resize(data, (500,500))
    ax.imshow(data, cmap = 'gray')
    ax.set_title('{}'.format(combined_training_set[['Negative for Pneumonia', 'Typical Appearance',
           'Indeterminate Appearance', 'Atypical Appearance']].idxmax(axis = 1)[Index_num]))

# Getting numpy array from dicom images
- This numpy array can then be used as an input to any tensorflow or pytorch deep learning model

In [ ]:
# currently ignoring the dicom image errors --------------> need help on that, please leave a comment if you can find a solution to it. Thanks in advance.
def getNumpyArraysSet(combined_training_set, ImageSize = (128,128)):
    X_temp = []
    Y_temp = []
    dicom_error_images_path = []
    i = 0
    for n in range(len(combined_training_set)):
        print('Percentage task complete {}'.format(n/len(combined_training_set) * 100))
        try:
            data = read_xray(combined_training_set.abs_image_path[n][0])
            try:
                Y_temp.append(combined_training_set[['Negative for Pneumonia', 'Typical Appearance',
                   'Indeterminate Appearance', 'Atypical Appearance']].iloc[n].values)
            except:
                pass
        except RuntimeError:
            i = i+1
            dicom_error_images_path.append(combined_training_set.abs_image_path[n][0])
            print('The number of images not read because of pydicom library errors are {}'.format(i))
    #     Get data and directory of images giving dicom error.
            continue
        data = cv2.resize(data, ImageSize)
        X_temp.append(data)
    # stacking up list in ndarray
    print('stacking arrays in list started')
    X = np.dstack(X_temp)
    try:
        y = np.dstack(Y_temp)
    except:
        print('y has no elemnts')
    print('Stacking finished')
    np.save('ImageArray'+str(ImageSize[0]), X)
    try:
        np.save('TargetArray'+str(ImageSize[0]), y)
    except:
        print('y has no elemnts')
    # Saving path to images that were not analysed.....
    df = pd.DataFrame(dicom_error_images_path)
    df.to_csv('PathDicomErrorImages.csv')
    print('All Done --------> Enjoy !!!!')
    return X, y
    
##################### 
# Getting the numpy arrays for training data set. 
X, Y = getNumpyArraysSet(combined_training_set, ImageSize = (128,128))
####################

# Getting Numpy Array From Test Data Set

In [ ]:
X = np.load('ImageArray128.npy')
y = np.load('TargetArray128.npy')

In [ ]:
directory = '../input/siim-covid19-detection/test'
# getting all the image paths
all_abs_file_paths = []
for dirpath,_,filenames in os.walk(directory):
        for f in filenames:
            all_abs_file_paths.append(os.path.abspath(os.path.join(dirpath, f)))

test_df = pd.DataFrame(all_abs_file_paths, columns = ['abs_image_path'])
test_df['id'] = test_df['abs_image_path'].apply(lambda x: x.split("/")[5])
test_df['abs_image_path'] = test_df['abs_image_path'].apply(lambda x: [x])
X_final_test = getNumpyArraysSet(test_df) # test data by Kaggle

# Data Preparation For Training the Model
- Once you have have numpy arrays you canuse any model or train the data, We will be using efficientNet to train our first model. 
- Hoever before we move forward we must prepare the numpy arrays for the pretrained model.

## Minimal data augmentation for upsampling the minority class

In [ ]:
transform_train = transforms.Compose([transforms.ToPILImage(),
                    transforms.RandomApply([torchvision.transforms.RandomRotation(10),transforms.RandomHorizontalFlip()],0.7), 
                    transforms.ToTensor()])

In [ ]:
#upsampling the minority class
newX = []
newY = []
for j in range(X.shape[-1]):
    if y[0,-1,j] == 1:
        for k in range(8):
            newX.append(transform_train(X[:,:,j]))
            newY.append(y[0,:,j])
    else:
        newX.append(transform_train(X[:,:,j]))
        newY.append(y[0,:,j])
balancedX = np.stack(newX)
balancedX = np.repeat(balancedX, repeats = 3, axis =1)
balancedY = np.stack(newY)

In [ ]:
# checking unique values in new dataset.
target_df = pd.DataFrame(balancedY, columns = ['Negative for Pneumonia', 'Typical Appearance',
           'Indeterminate Appearance', 'Atypical Appearance'])
target_df.value_counts()

## Splitting into training, test and Validation set

In [ ]:
## Splitting into training and validation set.
validFrac = 0.1   # Define the fraction of images to move to validation dataset
testFrac = 0.1    # Define the fraction of images to move to test dataset
validList = []
testList = []
trainList = []

for i in range(len(balancedX)):
    rann = np.random.random() # Randomly reassign images
    if rann < validFrac:
        validList.append(i)
    elif rann < testFrac + validFrac:
        testList.append(i)
    else:
        trainList.append(i)
        
nTrain = len(trainList)  # Count the number in each set
nValid = len(validList)
nTest = len(testList)
print("Training images =",nTrain,"Validation =",nValid,"Testing =",nTest)

## Converting numpy arrays into torch tensors......... Data that is understood by tensorflow and pytorch

In [ ]:
trainIds = torch.tensor(trainList)    # Slice the big image and label tensors up into
validIds = torch.tensor(validList) 
testIds = torch.tensor(testList)       #       training, validation, and testing tensors
trainX = torch.tensor(balancedX[trainIds,:,:,:])
trainY = torch.tensor(balancedY[trainIds,:])
validX = torch.tensor(balancedX[validIds,:,:,:])
validY = torch.tensor(balancedY[validIds,:])
testX = torch.tensor(balancedX[testIds,:,:,:])
testY = torch.tensor(balancedY[testIds,:])

# Loading the EfficientNet Model

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b2', num_classes=4) # loading model
model.to(dev) # Sendeng the model to GPU

In [ ]:
# Explore the model complexity
print(summary(model, input_size=(3, 128, 128)))

# Training the Model

In [ ]:
numClass = len(balancedY[:,0])
learnRate = 0.01          # Define a learning rate.
maxEpochs = 20            # Maximum training epochs
t2vRatio = 1.5            # Maximum allowed ratio of validation to training loss
t2vEpochs = 3            # Number of consecutive epochs before halting if validation loss exceeds above limit
batchSize = 128           # Batch size. Going too large will cause an out-of-memory error.
trainBats = nTrain // batchSize       # Number of training batches per epoch. Round down to simplify last batch
validBats = nValid // batchSize       # Validation batches. Round down
testBats = -(-nTest // batchSize)     # Testing batches. Round up to include all
opti = om.SGD(model.parameters(), lr = learnRate)   # Initialize an optimizer
lossEval = nn.CrossEntropyLoss()

for i in range(maxEpochs):
    model.train()                     # Set model to training mode
    epochLoss = 0.
    permute = torch.randperm(nTrain)  # Shuffle data to randomize batches
    trainX = trainX[permute,:,:,:]
    trainY = trainY[permute]
    for j in range(trainBats):        # Iterate over batches
        if j%20 == 0:
          print('The batch num is {}'.format(j))
        opti.zero_grad()              # Zero out gradient accumulated in optimizer
        batX = trainX[j*batchSize:(j+1)*batchSize,:,:,:].to(dev)   # Slice shuffled data into batches
        batY = trainY[j*batchSize:(j+1)*batchSize].to(dev)         # .to(dev) moves these batches to the GPU
        yOut = model(batX)            # Evalute predictions
        # print(yOut, batY)
        loss = F.cross_entropy(yOut, torch.max(batY, 1)[1])        # Compute loss
        epochLoss += loss.item()      # Add loss
        loss.backward()               # Backpropagate loss
        opti.step()                   # Update model weights using optimizer
    validLoss = 0.
    permute = torch.randperm(nValid)  # We go through the exact same steps, without backprop / optimization
    validX = validX[permute,:,:,:]    # in order to evaluate the validation loss
    validY = validY[permute]
    model.eval()                      # Set model to evaluation mode
    with torch.no_grad():             # Temporarily turn off gradient descent
        for j in range(validBats):
            opti.zero_grad()
            batX = validX[j*batchSize:(j+1)*batchSize,:,:,:].to(dev)
            batY = validY[j*batchSize:(j+1)*batchSize].to(dev)
            yOut = model(batX)
            validLoss += F.cross_entropy(yOut, torch.max(batY, 1)[1]).item()
    epochLoss /= trainBats            # Average loss over batches and print
    validLoss /= validBats
    print("Epoch = {:-3}; Training loss = {:.4f}; Validation loss = {:.4f}".format(i,epochLoss,validLoss))
    if validLoss > t2vRatio * epochLoss:
        t2vEpochs -= 1                # Test if validation loss exceeds halting threshold
        if t2vEpochs < 1:
            print("Validation loss too high; halting to prevent overfitting")
            break

# Testing the model on Test Data

In [ ]:
confuseMtx = np.zeros((numClass,numClass),dtype=int)    # Create empty confusion matrix
model.eval()
predictions = []
with torch.no_grad():
    permute = torch.randperm(nTest)                     # Shuffle test data
    testX = testX[permute,:,:,:]
    testY = testY[permute]
    for j in range(testBats):                           # Iterate over test batches
        batX = testX[j*batchSize:(j+1)*batchSize,:,:,:].to(dev)
        batY = testY[j*batchSize:(j+1)*batchSize].to(dev)
        yOut = model(batX) # Pass test batch through model
        for j in yOut:
            print(j, len(j),np.zeros(len(j)), torch.max(j,0)[1])
            b = np.zeros(len(j))
            index = torch.max(j,0)[1]
            b[index] = 1
            predictions.append(b)
        pred = yOut.max(1,keepdim=True)[1]              # Generate predictions by finding the max Y values
        for j in torch.cat((batY.max(1,keepdim=True)[1], pred),dim=1).tolist(): # Glue together Actual and Predicted to
            confuseMtx[j[0],j[1]] += 1                  # make (row, col) pairs, and increment confusion matrix
correct = sum([confuseMtx[i,i] for i in range(numClass)])   # Sum over diagonal elements to count correct predictions
print("Correct predictions: ",correct,"of",nTest)
print("Confusion Matrix:")
print(confuseMtx)
# print(classNames)

In [ ]:
testX

In [ ]:
# Get confusion matrix using scikit-learn
from sklearn.metrics import confusion_matrix
predictions = np.stack(predictions)
confusion_matrix()